### TFG-Aplicación de ténicas de Deep Learning para la predicción de demanda en medios de transporte

*Autor: Pedro Jesús Motos Nicolás* 

*Tutor: Fernando Terroso Sáenz*



 *-Preparación de librerías y constantes que vamos a utilizar más adelante*

In [1]:
#LIBRERIAS UTILIZADAS
import numpy as np
import pandas as pd
import csv
from IPython.display import display, HTML

In [2]:
#CONSTANTES BICICLETAS
DOC_1_BICI='Archivos/NY_clean_city_bike_2019-01.csv'
DOC_2_BICI='Archivos/NY_clean_city_bike_2019-02.csv'
DOC_3_BICI='Archivos/NY_clean_city_bike_2019-03.csv'
DOC_4_BICI='Archivos/NY_clean_city_bike_2019-04.csv'
DOC_5_BICI='Archivos/NY_clean_city_bike_2019-05.csv'
DOC_6_BICI='Archivos/NY_clean_city_bike_2019-06.csv'

COLUMN_BICI='starttime'

#CONSTANTES TAXIS
DOC_1_TAXI='Archivos/yellow_clean_tripdata_2019-01.csv'
DOC_2_TAXI='Archivos/yellow_clean_tripdata_2019-02.csv'
DOC_3_TAXI='Archivos/yellow_clean_tripdata_2019-03.csv'
DOC_4_TAXI='Archivos/yellow_clean_tripdata_2019-04.csv'
DOC_5_TAXI='Archivos/yellow_clean_tripdata_2019-05.csv'
DOC_6_TAXI='Archivos/yellow_clean_tripdata_2019-06.csv'

COLUMN_TAXI='pickup_datetime'


In [3]:
##Introducir en la memoria los registros que estamos teniendo 
yelp = pd.read_csv(DOC_1_TAXI)
yelp1 = pd.read_csv(DOC_2_TAXI)
yelp2 = pd.read_csv(DOC_3_TAXI)
yelp3 = pd.read_csv(DOC_4_TAXI)
yelp4 = pd.read_csv(DOC_5_TAXI)
yelp5 = pd.read_csv(DOC_6_TAXI)

In [4]:
yelp_b = pd.read_csv(DOC_1_BICI)
yelp1_b = pd.read_csv(DOC_2_BICI)
yelp2_b = pd.read_csv(DOC_3_BICI)
yelp3_b = pd.read_csv(DOC_4_BICI)
yelp4_b = pd.read_csv(DOC_5_BICI)
yelp5_b = pd.read_csv(DOC_6_BICI)

In [5]:
##Media de registros que estamos utilizando
MediaTaxi=(len(yelp)+len(yelp1)+len(yelp2)+len(yelp3)+len(yelp4)+len(yelp5))/6
MediaBici=(len(yelp_b)+len(yelp1_b)+len(yelp2_b)+len(yelp3_b)+len(yelp4_b)+len(yelp5_b))/6

print("Media de Taxis -->",MediaTaxi,"registros." ,"\nMedia de Bicis -->",MediaBici,"registros." )

Media de Taxis --> 7303374.833333333 registros. 
Media de Bicis --> 1465401.0 registros.


### Función para la lectura de datos



*Esta función recibe el fichero .csv, con los diferentes registros de viajes realizados por los usuarios, y también, la  columna de la fecha en la que se realizó el viajes, que es con la que vamos a trabajar.*

*Transforma estos datos en un DataFrame con las columnas, "ANIO, MES, DIA, HORA, MIN, SEG, Fecha"*

In [6]:
def RecopilacionDatos2(Archivo, x):

    total= []
    for gm_chunk in pd.read_csv(Archivo,chunksize=400000):
        for c in gm_chunk[x]:

            d0 = c.split()
            result = d0

            #print(d0)     #formato salida del result = (['2019-01-01', '00:46:40'])
            d1 = result[0].split('-')
            d2 = result[1].split(':')
            #print(d2)
            result = d1+d2
            #print(result[1])

            total.append(result) #lo introducimos dentro de nuestro array
            #print(total)

    ecom = pd.DataFrame(total,columns = ["ANIO","MES","DIA","HORA","MIN","SEG"])   #index --> para nombrar a las filas
    ecom['concatenated'] = ecom['MES'] +'-'+ ecom['DIA'] +'#'+ ecom['HORA']
    ecom.columns = ['ANIO','MES','DIA','HORA','MIN','SEG','Fecha']
    ecom
        
    return(ecom)

### Función para la transformación de datos


*Esta función recibe como entrada el dataframe creado en la función anterior con la fecha de la realización del viaje segmentada en columnas*

*Elimina las columnas innecesarias y transforma aquellos datos que nos hacen falta. Además también realiza la suma de los viajes 
que se han realizado un determinado dia y a una hora concreta*

In [7]:
def TratamientoDatos(dataF):
    
    a = dataF['Fecha'].value_counts().sort_index()
    a = pd.DataFrame(a)
    a = a.reset_index()
    a.columns = ['Fecha','Numero de viajes']
    a
    
    subtotal=[]

    for indice_fila, fila in a.iterrows():
        f=fila[0], fila[1]
        s=f[0].split('#')
        s.append(f[1])

        subtotal.append(s)

    result = pd.DataFrame(subtotal)
    result.columns = ['Fecha','Hora','Numero de viajes']
    
    return(result)

### Introducción y tratamiento de datos


*Una vez que se han explicado las funciones pasamos a su utilización. Estas se utilizarán para los primeros seis meses del año 2019*



#### Viajes en Taxi


In [8]:
##Datos de Enero
Enero = RecopilacionDatos2(DOC_1_TAXI,COLUMN_TAXI)
Enero = Enero.drop(Enero[Enero['MES']!='01'].index)
Enero = TratamientoDatos(Enero)
#Enero

In [9]:
##Datos de Febrero
Febrero = RecopilacionDatos2(DOC_2_TAXI,COLUMN_TAXI)
Febrero = Febrero.drop(Febrero[Febrero['MES']!='02'].index)
Febrero = TratamientoDatos(Febrero)
#Febrero

In [10]:
##Datos de Marzo
Marzo = RecopilacionDatos2(DOC_3_TAXI,COLUMN_TAXI)
Marzo = Marzo.drop(Marzo[Marzo['MES']!='03'].index)
Marzo = TratamientoDatos(Marzo)
#Marzo

In [11]:
##Datos de Abril
Abril = RecopilacionDatos2(DOC_4_TAXI,COLUMN_TAXI)
Abril = Abril.drop(Abril[Abril['MES']!='04'].index)
Abril = TratamientoDatos(Abril)
#Abril

In [12]:
##Datos de Mayo
Mayo = RecopilacionDatos2(DOC_5_TAXI,COLUMN_TAXI)
Mayo = Mayo.drop(Mayo[Mayo['MES']!='05'].index)
Mayo = TratamientoDatos(Mayo)
#Mayo

In [13]:
##Datos de Junio
Junio = RecopilacionDatos2(DOC_6_TAXI,COLUMN_TAXI)
Junio = Junio.drop(Junio[Junio['MES']!='06'].index)
Junio = TratamientoDatos(Junio)
#Junio

*Concatenamiento de todos los meses en una tabla única (viajes en taxi)*

In [14]:
Total_column_TAXI = pd.concat([Enero, Febrero, Marzo, Abril, Mayo, Junio]).reset_index()   
Total_column_TAXI = Total_column_TAXI.drop(['index'], axis=1)
#Total_column_TAXI

In [15]:
#Comprobación de que los meses se han introducido correctamente
#display(HTML(Total_column_TAXI.to_html()))

#### Viajes en Bicicleta

In [16]:
#Datos de Enero
Enero = RecopilacionDatos2(DOC_1_BICI,COLUMN_BICI)
Enero = Enero.drop(Enero[Enero['MES']!='01'].index)
Enero = TratamientoDatos(Enero)
#Enero

In [17]:
#Datos de Febrero
Febrero = RecopilacionDatos2(DOC_2_BICI,COLUMN_BICI)
Febrero = Febrero.drop(Febrero[Febrero['MES']!='02'].index)
Febrero = TratamientoDatos(Febrero)
#Febrero

In [18]:
#Datos de Marzo
Marzo = RecopilacionDatos2(DOC_3_BICI,COLUMN_BICI)
Marzo = Marzo.drop(Marzo[Marzo['MES']!='03'].index)
Marzo = TratamientoDatos(Marzo)
#Marzo

In [19]:
#Datos de Abril
Abril = RecopilacionDatos2(DOC_4_BICI,COLUMN_BICI)
Abril = Abril.drop(Abril[Abril['MES']!='04'].index)
Abril = TratamientoDatos(Abril)
#Abril

In [20]:
#Datos de Mayo
Mayo = RecopilacionDatos2(DOC_5_BICI,COLUMN_BICI)
Mayo = Mayo.drop(Mayo[Mayo['MES']!='05'].index)
Mayo = TratamientoDatos(Mayo)
#Mayo

In [21]:
#Datos de Junio
Junio = RecopilacionDatos2(DOC_6_BICI,COLUMN_BICI)
Junio = Junio.drop(Junio[Junio['MES']!='06'].index)
Junio = TratamientoDatos(Junio)
#Junio

*Concatenamiento de todos los meses en una tabla única (Viajes en  bicicleta)*

In [22]:
Total_column_BICI = pd.concat([Enero, Febrero, Marzo, Abril, Mayo, Junio]).reset_index()  
Total_column_BICI = Total_column_BICI.drop(['index'], axis=1)
#Total_column_BICI

In [23]:
#Comprobación de que los meses se han introducido correctamente
#display(HTML(Total_column_BICI.to_html()))

### Funciones para la unión de taxis y bicis



*Una vez que tenemos ambos tipos de datos en tablas diferentes tenemos que unificarlos en una sola tabla, para asi poder trabajar con ellos.*



 *- En esta función se crea una columna que nos hara falta eliminan aquellas columnas que no nos hacen falta en este momento*

In [24]:
def fusion1(dataF):
    
    dataF['key_column'] = dataF['Fecha']+'#'+dataF['Hora']      #Hago una nueva columna con la fecha y hora concatenada para que sea la columna llave
    dataF=dataF.drop(['Fecha','Hora'], axis=1)                  #Eliminamos columnas que no nos sirven    
    
    return(dataF)

 *- Función que realiza la unión de las dos tabla. Como resultado obtendremos las tablas de los viajes realizados en bicicleta y en taxique unificadas.* 
 
 *Además, se comprueba que ambas tablas tengan el mismo número de viajes y las mismas horas de trabajo, tanto en los taxis como en las bicicletas. Esto es muy importante, para que no haya perdida de horarios, (y por lo tanto, de datos) en el caso de que un transporte no haya trabajado algún día u hora especifica (por el motivo que sea).*


In [25]:
def fusion2(dataF1,dataF2):
    '''
    -Comprobación de que tenemos las mismas horas y dias en ambos df
    -Unión del numero de viajes de taxi y bici a cada hora#fecha correspondiente
    '''
    x1=fusion1(dataF1)
    x1_bis=x1.drop('Numero de viajes', axis=1)
    
    x2=fusion1(dataF2)
    x2_bis=x2.drop('Numero de viajes', axis=1)
    
    Total_key_column = pd.concat([x1_bis, x2_bis]).drop_duplicates().sort_values('key_column')
    Total_key_column =Total_key_column.reset_index()
    Total_key_column =Total_key_column[['key_column']]
    
    
    join_x1=Total_key_column.join(x1.set_index('key_column'), on='key_column')
    join_x1.rename(columns={'Numero de viajes': 'Viajes taxi'},inplace=True)    ##Cambiamos el nombre de la columna para que no colapse al unirlas
    join_x2=Total_key_column.join(x2.set_index('key_column'), on='key_column')
    join_x2.rename(columns={'Numero de viajes': 'Viajes bici'},inplace=True)
    result_total = join_x1.join(join_x2.set_index('key_column'), on='key_column').fillna(0)   
    
    return(result_total)
        

#### Unificación de tablas

In [26]:
TABLA_UNION=fusion2(Total_column_TAXI,Total_column_BICI)
TABLA_UNION

,key_column,Viajes taxi,Viajes bici
0,01-01#00,13256,129
1,01-01#01,15205,260
2,01-01#02,12984,273
3,01-01#03,10221,179
4,01-01#04,6670,117
...,...,...,...
4338,06-30#19,7960,4122
4339,06-30#20,7565,3385
4340,06-30#21,7856,2548
4341,06-30#22,7428,1798


### Preparación de los datos para la red neuronal


*Una vez que tenemos las dos tablas unidas podemos preparar los datos para que sean introducidos en nuestra red neuronal, esto es lo que hemos realizado en la siguiente función*

*-Esta función recibe como entrada un Dataframe. Devuelve otro datraframe, pero con los datos en el formato adecuado para su introducción en la red neuronal, de la siguiente forma: "Hora, Viajes en taxi, Viajes_en_bici"*


In [27]:
def PreparacionDatosRNA(dataF):
    '''
    -Eliminación de la columna clave
    -Introduccion de Hora y Fecha
    '''
    a=dataF
    subtotal=[]

    for indice_fila, fila in a.iterrows():
        f=fila[0], fila[1], fila[2]
        s=f[0].split('#')
        s.append(f[1])    #Aniadimos columna de viajes de taxi
        s.append(f[2])    #Aniadimos columna de viajes de bici

        subtotal.append(s)

    result = pd.DataFrame(subtotal)
    result.columns = ['Fecha','Hora','Viajes_en_taxi','viajes_en_bici']
    
    return(result)

#### Preparación de los datos
*Una vez que se han explicado las funciones pasamos a su utilización.*

In [28]:
DatosLSTM=PreparacionDatosRNA(TABLA_UNION)
DatosLSTM=DatosLSTM.drop(['Fecha'], axis=1)
DatosLSTM

,Hora,Viajes_en_taxi,viajes_en_bici
0,00,13256,129
1,01,15205,260
2,02,12984,273
3,03,10221,179
4,04,6670,117
...,...,...,...
4338,19,7960,4122
4339,20,7565,3385
4340,21,7856,2548
4341,22,7428,1798


#### Introducción de los datos en un archivo .csv para la RNA

In [29]:
DatosLSTM.to_csv('Datos_LSTM.csv', index=None)